# Volcado Tabular de Vencimientos Explained con Otros Campos

In [77]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [78]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [79]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [80]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [81]:
fecha_proceso = date(2025, 7, 4)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [82]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [83]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [84]:
explained[0]

{'dealNumber': '2256',
 'legNumber': 1,
 'tipoPata': 'OVERNIGHT_INDEX',
 'recPay': 'A',
 'fechaInicial': '2025-01-06',
 'fechaFinal': '2025-07-04',
 'fechaInicialIndice': '2025-01-06',
 'fechaFinalIndice': '2025-07-04',
 'fechaPago': '2025-07-04',
 'nocional': 3000000000.0,
 'amortizacion': 0.0,
 'amortEsFlujo': True,
 'monedaNocional': 'CLP',
 'nombreIndice': 'ICPCLP',
 'valorIndiceInicial': 24801.68,
 'valorIndiceFinal': 25425.95,
 'valorTasa': 0.0506,
 'tipoTasa': 'LinAct360',
 'interes': 75478333.0,
 'flujo': 75478333.0,
 'spread': 0.0,
 'gearing': 1.0,
 'numDecimalesTasaEquivalente': 4,
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaValorTasaEquivalente': 'round((valorIndiceFinal / valorIndiceInicial - 1) * 1 / fraccionAgno, numDecimalesTasaEquivalente)',
 'formulaInteres': 'nocional * (formulaValorTasaEquivalente + spread) * fraccionAgno'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [85]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [86]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'COB_TASA_INT_ACTIVOS',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': {'regulatory_portfolio': 'COBERTURA_VALOR_RAZONABLE'}}

In [87]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [88]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
# TRY - EXCEPT
#del df_explained_ok["formulaInteresMonedaPago"]
#del df_explained_ok["formulaAmortizacionMonedaPago"]

In [89]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaInicialIndice,fechaFinalIndice,fechaPago,nocional,...,interes,flujo,spread,gearing,numDecimalesTasaEquivalente,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,2256,1,OVERNIGHT_INDEX,A,2025-01-06,2025-07-04,2025-01-06,2025-07-04,2025-07-04,3.000000e+09,...,7.547833e+07,7.547833e+07,0.0,1.0,4.0,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
1,2256,2,FIXED_RATE,P,2025-01-06,2025-07-04,NaN,NaN,2025-07-04,-3.000000e+09,...,-6.018875e+07,-6.018875e+07,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
2,2257,1,FIXED_RATE,A,2025-01-06,2025-07-04,NaN,NaN,2025-07-04,7.800000e+09,...,1.477645e+08,1.477645e+08,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
3,2257,2,OVERNIGHT_INDEX,P,2025-01-06,2025-07-04,2025-01-06,2025-07-04,2025-07-04,-7.800000e+09,...,-1.962437e+08,-1.962437e+08,0.0,1.0,4.0,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
4,4685,1,OVERNIGHT_INDEX,A,2025-01-06,2025-07-04,2025-01-06,2025-07-04,2025-07-04,4.300000e+09,...,1.081856e+08,1.081856e+08,0.0,1.0,4.0,SWAP,NO,SWAP_ICP,C,NEGOCIACION
5,4685,2,FIXED_RATE,P,2025-01-06,2025-07-04,NaN,NaN,2025-07-04,-4.300000e+09,...,-1.452809e+08,-1.452809e+08,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
6,6206,1,OVERNIGHT_INDEX,A,2025-01-06,2025-07-04,2025-01-06,2025-07-04,2025-07-04,9.400000e+09,...,2.364988e+08,2.364988e+08,0.0,1.0,4.0,SWAP,NO,SWAP_ICP,C,NEGOCIACION
7,6206,2,FIXED_RATE,P,2025-01-06,2025-07-04,NaN,NaN,2025-07-04,-9.400000e+09,...,-2.435096e+08,-2.435096e+08,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION


In [ ]:
df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [91]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [92]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [93]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

KeyError: 'flujoMonedaPago'

Aplicar conversiones de nombres de producto

In [ ]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [ ]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [ ]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [ ]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (6, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       2267  Swap Promedio Cámara  2025-07-10                   C   
1       2775           Swap Moneda  2025-07-10                   C   
2       5667            Swap Tasas  2025-07-10                   C   
3       6231  Swap Promedio Cámara  2025-07-10                   C   
4       6232  Swap Promedio Cámara  2025-07-10                   C   

  hedgeAccounting                regulatoryPortfolio  pataActivaAmortizacion  \
0              NO                        NEGOCIACION                    0.00   
1              NO                        NEGOCIACION                  164.38   
2              NO                        NEGOCIACION                    0.00   
3              NO  COBERTURA_RIESGO_POSICION_BALANCE                    0.00   
4              NO                        NEGOCIACION                    0.00   

  pataActivaMonedaNocional pataActivaFechaPag

In [ ]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,2267,Swap Promedio Cámara,2025-07-10,C,NO,NEGOCIACION,0.00,CLP,2025-07-11,8.649789e+07,0.0,CLP,2025-07-11,-6.914703e+07,1.735086e+07
1,2775,Swap Moneda,2025-07-10,C,NO,NEGOCIACION,164.38,CLF,2025-07-10,7.684415e+06,-4693215.0,CLP,2025-07-10,-6.808838e+06,8.755774e+05
2,5667,Swap Tasas,2025-07-10,C,NO,NEGOCIACION,0.00,USD,2025-07-10,1.393083e+05,0.0,USD,2025-07-10,-1.440640e+05,-4.755634e+03
3,6231,Swap Promedio Cámara,2025-07-10,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.00,CLP,2025-07-11,4.452097e+07,0.0,CLP,2025-07-11,-4.390507e+07,6.159026e+05
4,6232,Swap Promedio Cámara,2025-07-10,C,NO,NEGOCIACION,0.00,CLP,2025-07-11,2.437467e+08,0.0,CLP,2025-07-11,-2.442293e+08,-4.826663e+05
5,6233,Swap Promedio Cámara,2025-07-10,C,NO,NEGOCIACION,0.00,CLP,2025-07-11,2.442293e+08,0.0,CLP,2025-07-11,-2.437467e+08,4.826663e+05


In [ ]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\{f_proc}_deal_level.xlsx", index=False)
